In [2]:
import pandas as pd
import os

In [3]:
os.chdir('../')

In [5]:
data=pd.read_csv('artifacts\data_ingestion\winequality-red.csv')
# Get column names and data types
column_info = data.dtypes.to_dict()

# Print the column information in key-value format
for column, dtype in column_info.items():
    print(f"{column}: {dtype}")

fixed acidity: float64
volatile acidity: float64
citric acid: float64
residual sugar: float64
chlorides: float64
free sulfur dioxide: float64
total sulfur dioxide: float64
density: float64
pH: float64
sulphates: float64
alcohol: float64
quality: int64


In [6]:
#entity data validation
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict
    

In [7]:
#configuration
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema=self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema, 
        )

        return data_validation_config

In [8]:
#components

import os
from src.mlProject import logger

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e
##if everything matches in the schema yaml , status file will turn true  

In [9]:
#pipeline

#7. update pipeline in data validation

try:
    config=ConfigurationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-08-07 03:46:29,798: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-07 03:46:29,802: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-07 03:46:29,810: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-07 03:46:29,813: INFO: common: created directory at: artifacts]
[2023-08-07 03:46:29,816: INFO: common: created directory at: artifacts/data_validation]
